# 3- Administrador de agentes

Este código permite la creación de un sistema para gestionar y coordinar diferentes agentes. En el ejemplo se muestra la capacidad del coordinador para escoger el mejor agente para realizar una tarea especifica.

Esencialmente, `AgentManager` actúa como un coordinador, interpretando los mensajes de los usuarios, eligiendo el agente más adecuado para responder a esa solicitud, y pasando el mensaje a ese agente para su procesamiento. También gestiona cualquier interacción subsiguiente entre los agentes y el usuario, manejando la memoria compartida del diálogo para mantener un contexto coherente durante la conversación.

## 3.1 - Importaciones y configuraciones

In [1]:
import os
from typing import Optional
from openai import OpenAI
from dotenv import load_dotenv
from abc import ABC, abstractmethod
import json
from exec_code import execute_code, extract_code_from_string
from prompt import CODE_DEVELOPER_SYSTEMPROMPT, DOCUMENTATION_DEVELOPER_SYSTEMPROMPT, COORDINATOR_SYSTEMPROMPT

load_dotenv()

client = OpenAI()

## 3.2 - Definición de clases

In [2]:
class Agent(ABC):
    
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None
    ) -> None:
        
        model = model if model else os.getenv("OPENAI_DEFAULT_MODEL")
        
        self.memory = []
        self.model = model
        
        if system_prompt:
            self.memory.append({"role": "system", "content": system_prompt})
    
    def add_to_memory(self, role, message):
        self.memory.append({"role": role, "content": message})

    def get_schema(self):
        return self.function
        
    @abstractmethod
    def run(self,prompt):
        """User must define this method. Run the agent"""
    
class ConversationAgent(Agent):
    
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None,
        agent_name: Optional[str] = None,
        description: Optional[str] = None
    ) -> None:
        super().__init__(model,system_prompt)

        self.agent_name = agent_name
        self.description = description

        self.function={
            "name": self.agent_name,
            "description": self.description,
            "parameters": {
                "type": "object",
                "properties": {
                    "user_prompt": {
                        "type": "string",
                        "description": "User prompt to the agent"
                    }
                },
                "required": ["user_query"]
            }
        }
        
    def run(self,prompt):

        self.add_to_memory("user", prompt)
        completion=''
        stream = client.chat.completions.create(
            messages=self.memory,
            model=self.model,
            stream=True
        )
        print(f"\n{self.agent_name}:")
        for chunk in stream:
            text_chunk=chunk.choices[0].delta.content
            if text_chunk:
                completion+=text_chunk
                print(text_chunk, end='', flush=True)
        
        self.add_to_memory("assistant", completion)
        
        return completion

## 3.3 - Configuración del AgentManager

El `AgentManager` es una clase que hereda de la clase abstracta `Agent`. Esta clase está diseñada para orquestar y manejar las interacciones entre varios agentes conversacionales, como `CodeDeveloper`, `ImproveDeveloper` y `DocumentationDeveloper`. Cada uno de estos agentes tiene funciones especializadas, como generación de código, diseño de interfaces de usuario, y creación de documentación, respectivamente.

Aquí están los elementos clave de la clase `AgentManager`:

- **run**: El método `run` es donde se lleva a cabo la lógica principal del `AgentManager`. Cuando se llama a `run` con un mensaje de usuario:
  1. Añade el mensaje a la memoria del `AgentManager`.
  2. Decide cuál de los agentes debe manejar esta solicitud basándose en su comprensión del mensaje y la lista de agentes disponibles.
  3. Basándose en esta información, selecciona el agente adecuado y le pasa la solicitud.
  4. El método continúa y ejecuta el agente seleccionado, esperando su respuesta. Si la respuesta contiene código para ejecutar, intenta ejecutar este código.

In [3]:


class AgentManager(Agent):
    
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None,
        list_agents: Optional[list] = None
    ) -> None:
        super().__init__(model,system_prompt)
        
        self.list_agents=list_agents
    
    
    def get_agent_list_schemas(self):
        return [agent.get_schema() for agent in self.list_agents]   
    
    def run(self,prompt):
        
        print(f"----------------- RUN MANAGER -----------------")
        print(f"User: {prompt}")
                            
        # Choose what agent i have to execute to resolve the problem (Select speaker)
        ######################################
        messages = self.memory + [{"role": "user", "content": f"[PROMPT]\n{prompt}\n\n[AGENTS]\n{self.get_agent_list_schemas()}"}]
        chat_completion = client.chat.completions.create(
            messages=messages,
            model=self.model,
            functions=self.get_agent_list_schemas()
        )
                
        # Get the agent to execute
        agent_name=chat_completion.choices[0].message.function_call.name
        agent_prompt=json.loads(chat_completion.choices[0].message.function_call.arguments)['user_prompt']
         
        #get the class agent to execute
        class_agent = next((agent for agent in self.list_agents if agent.agent_name == agent_name), None)
        
        # Save agents and remove the agent choosen to another list. It is use to store memory in the future in all agents without the agent choosen
        other_agents = [agent for agent in self.list_agents if agent.agent_name != agent_name]
        
        # Add the prompt to the agent manager memory and the others agents
        ######################################
        self.add_to_memory("user", prompt)
        # add memory to others agents
        for agent in self.list_agents:
            agent.add_to_memory("user", agent_prompt)
            
        #Execute the algorithm
        ######################################
        for i in range(3):
            
            # # Run agent
            ######################################
            response_agent=class_agent.run(f"{class_agent.agent_name}: {agent_prompt}")
            # add memory to others agents
            for agent in other_agents:
                agent.add_to_memory("user", f"{class_agent.agent_name}: {agent_prompt}")
            
            # if we have code to execute, execute it
            ######################################
            code_to_execute = extract_code_from_string(response_agent)
            if code_to_execute:
                try:
                    print("Executing code...")
                    exec_response = execute_code(code_to_execute,use_docker=False)
                    if exec_response[0] == 0:
                        print(f"Code executed successfully")
                        break
                    else:
                        print(f"The code: {code_to_execute}.\n\nError executing code: {exec_response[1]}")
                        agent_prompt=f"\n\nError executing code: {exec_response[1]}"
                except Exception as e:
                    print(f"The code: {code_to_execute}.\n\nError executing code: {e}")
                    agent_prompt=f"\n\nError executing code: {e}"
            else:
                print(f"No code found to execute.")
                agent_prompt=f"\n\nNo code found to execute."
        
        # Add the response from the agent manager to the memory
        ######################################
        self.add_to_memory("assistant", response_agent)
                
        return response_agent

## 3.4 - Configuración de los agentes

Los tres agentes, `CodeDeveloper`, `ImproveDeveloper` y `DocumentationDeveloper`, son subclases especializadas de `ConversationAgent`, cada una equipada para manejar diferentes tipos de solicitudes mediante el `system_prompt` que se haya configurado:

- **CodeDeveloper**: Genera código en respuesta a problemas o solicitudes de programación proporcionados por el usuario. Su propósito es interactuar con los usuarios que necesitan asistencia en la creación de scripts o software.

- **ImproveDeveloper**: Atiende solicitudes relacionadas con el el mejoramiento diseño. Este agente ofrece soluciones o propuestas de diseño para que las interfaces sean más efectivas y atractivas.

- **DocumentationDeveloper**: Se especializa en la creación de documentación para código o proyectos de software. Este agente ayuda a los usuarios a redactar una documentación clara y comprensible, importante para cualquier proyecto de desarrollo.

- **AgentManager**: Es el coordinador central que gestiona qué agente, desde su listado de agentes, debe responder a una solicitud específica. Interpreta los mensajes del usuario, selecciona al agente más adecuado basándose en la naturaleza del mensaje y la especialización del agente, y redirige la solicitud a ese agente para su procesamiento.

In [4]:
CodeDeveloper=ConversationAgent(
    model="gpt-4-1106-preview",
    agent_name="CodeDeveloper",
    description="Agent to create code in python",
    system_prompt=CODE_DEVELOPER_SYSTEMPROMPT
)

ImproveDeveloper=ConversationAgent(
    model="gpt-4-1106-preview",
    agent_name="ImproveDeveloper",
    description="Agent to resolve improvements in python",
    system_prompt=CODE_DEVELOPER_SYSTEMPROMPT
)

DocumentationDeveloper=ConversationAgent(
    model="gpt-4-1106-preview",
    agent_name="DocumentationDeveloper",
    description="Agent to document code",
    system_prompt=DOCUMENTATION_DEVELOPER_SYSTEMPROMPT
)

# Manager Agents
manager=AgentManager(
    model="gpt-4-1106-preview",
    list_agents=[
        CodeDeveloper,
        DocumentationDeveloper,
        ImproveDeveloper
    ],
    system_prompt=COORDINATOR_SYSTEMPROMPT
)

## 2.5- Ejecutar

In [5]:

manager.run("Create a code for a face detector using my camera, show me it in display and it must run by 30 sec and end.")

----------------- RUN MANAGER -----------------
User: Create a code for a face detector using my camera, show me it in display and it must run by 30 sec and end.


UnboundLocalError: cannot access local variable 'agent_prompt' where it is not associated with a value

In [ ]:
manager.run("Improve the display of the app with better visual design, such as counting faces and coloring the inner faces of the box and adding opacity to it.")

In [ ]:
manager.run("All code before, document the code and explain it")